In [41]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
import webscraping
import requests
import os
import re
import csv
#from selenium import webdriver
from langdetect import detect
import pandas as pd

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [23]:
UrlsFiles = open("urlpages.txt", "w")

for i in range(1, 101):
    url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, features='lxml')
    for a in soup.find_all('a', class_="bookTitle"):
        UrlsFiles.write(a.get('href')+'\n')

UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [24]:
headpart = "https://www.goodreads.com"

In [25]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [26]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [34]:
UrlsFiles = open("urlpages.txt", "r")

counter_pages = 0
counter_html = 0
for x in UrlsFiles:
    if counter_html % 100 == 0:
        counter_pages = counter_pages+1
        
    counter_html = counter_html + 1
    
    subdirectory = pathAncestor + "/page " + str(counter_pages)
    article_name = "/article_"+str(counter_html)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        try:
            page = requests.get(link)
        except:
            with open("failureRequest.txt", "wb") as err_file:
                err_file.write(link)
                err_file.close()
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [20]:
def scrap_book(tsv_writer, article):
    global bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url;
    
    with open(article, 'r', encoding="utf-8") as out_file:
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text

        #extract rating and review count
        try:
            ratings = soup.find_all('a', href="#other_reviews") #search the ratings in its
            rating_count = -1
            rating = -1
            for raiting in ratings:
                if raiting.find_all('meta', itemprop="ratingCount"):
                    ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
                elif raiting.find_all('meta', itemprop="reviewCount"):
                    reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        except:
            ratingCount = " "
            reviewCount = " "
            
        #extract the book title
        try:
            bookTitle = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()
        except:
            bookTitle = " "

        #extract the book authors
        try:
            bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0]
        except:
            bookAuthors = " "

        #extract the book authors, we shoul FIX it.
        try:
            Plot = soup.find_all('div', id="description")[0].contents[3].text
            if detect(Plot) != "en":
                Plot = " "
        except:
            try:
                Plot = soup.find_all('div', id="description")[0].contents[1].text
                if detect(Plot) != "en":
                    Plot = " "
            except:
                Plot = " "
                

        #extract the date
        try:
            date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
            Published = date[1]+" "+date[2]+" "+date[3]
        except:
            Published = " "

        #Rating Value
        try:
            ratingValue = soup.find('span', itemprop="ratingValue").text.strip()
        except:
            ratingValue = " "

        #Number of pages
        try:
            NumberofPages = soup.find('span', itemprop="numberOfPages").text.split()[0]
        except:
            NumberofPages = " "

        #Title series
        try:
            bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
        except:
            bookSeries = " "
            
        #Places
        try:
            Setting = []
            for places in soup.find_all('a', href= re.compile(r'/places/*')):
                Setting.append(places.text)
            Setting = ", ".join(Setting) if len(Setting)>=1 else " "
        except:
            Setting = " "

        #list of characters
        try:
            Characters = []
            for character in soup.find_all('a', href= re.compile(r'/characters/*') ):
                Characters.append(character.text)
            Characters = ", ".join(Characters) if len(Characters)>=1 else " "
        except:
            Characters = " "

        #extract the Url
        try:
            Url = soup.find_all('link')[0]["href"]
        except:
            Url = " "

        tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url])

In [22]:
path = "./html_pages"

filenames = os.listdir(path)
for i in range(1, 101):
    filenames = os.listdir(path + '/' + str(i))

    for file in filenames:
        with open(path + '/' + str(i) + './article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                            'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                            'Characters', 'Setting', 'Url'])
            scrap_book(tsv_writer, path + '/' + str(i) + "/" + file)

### creating dataset

In [8]:
path = "./html_pages"
suffix = ".tsv"
filenames = os.listdir(path)
data2 = pd.DataFrame()
for i in range(1, 101):
    filenames = os.listdir(path + '/' + str(i))
    for file in filenames:
        if file.endswith(suffix):
            with open(path + '/' + str(i) + '/article_'+str(file.split("_")[1]), 'r', encoding="utf-8", newline='') as out_file:
                    df = pd.read_csv(out_file,sep = "\t")
                    if  df.loc[0,"Plot"] != " " and df.loc[0,"bookTitle"] != " ":
                        data2 = pd.concat([data2,df])
with open("output1-100.tsv", "w", encoding="utf-8", newline="") as text_file: text_file.write(data2.to_csv(index=False))

## RQ[2] Search Engine

In [42]:
df1 = pd.read_csv('output1-100.tsv',sep=",")
#df2 = pd.read_csv('NEWoutput2.tsv', sep="\t")
#df3 = pd.read_csv('NEWoutput3.tsv', sep="\t")

#complete_dataset = pd.concat([df1, df2, df3])
#complete_dataset = complete_dataset.reset_index(drop=True) # reset index

#complete_dataset = complete_dataset.drop_duplicates(subset=['bookTitle'])
#complete_dataset = complete_dataset.dropna(subset=['Plot'])
#complete_dataset = complete_dataset.reset_index(drop=True)
#complete_dataset

## **Cleaning data**

In [43]:
#DEF Cleaning
import nltk

from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#from nltk.stem import WordNetLemmatizer

In [44]:
def cleaningDataset(df):
    for i, row in df.iterrows():
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        
        df.at[i, 'Plot'] = tokenizer.tokenize(df.at[i, 'Plot'].lower()) # remove the punctuation
        df.at[i, 'Plot'] = [w for w in df.at[i, 'Plot'] if not w in set(stopwords.words('english'))]  # words in english to avoid few data for the cleaning data
        df.at[i, 'Plot'] = [PorterStemmer().stem(word) for word in df.at[i, 'Plot']] # contesto
    return df

In [45]:
def createVocabulary(df):
    vocabulary = {}
    for i, row in df.iterrows():
        for word in df.at[i, 'Plot']:
            if word in vocabulary.keys():
                if "document_"+str(i) not in vocabulary[word]:
                    vocabulary[word].append("document_"+str(i))
            else:
                vocabulary[word] = ["document_"+str(i)]
    return vocabulary

In [46]:
#create and clean the dataset
df = df1[1:20].copy()#complete_dataset.copy()
df = cleaningDataset(df)    

In [47]:
vocabulary = createVocabulary(df)

In [48]:
#create file .csv
import csv

with open('./vocabulary.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Word", "Term_id", "Document_List"])
    
    list_of_words = vocabulary.keys()
    for id, word in enumerate(list_of_words, 1):
        term_id_i = "term_id_"+str(id)
        tsv_writer.writerow([word, term_id_i, vocabulary[word]]) 

## RQ[2.1.2] Execute the query

In [49]:
def cleanQuery(row):
    row = row.split(" ") # split the string query
    for element in row:
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        element = tokenizer.tokenize(element.lower()) # remove the punctuation
    
    row = [w for w in row if not w in set(stopwords.words('english'))] # words in english to avoid few data for the cleaning data
    
    #row = [lemmatizer.lemmatize(word) for word in row]
    row = [PorterStemmer().stem(word) for word in row]
    
    return row

In [60]:
query = input("Insert your query string: ")

Insert your query string: door end


In [61]:
cleanQString = cleanQuery(query)
cleanQString

['door', 'end']

open the dataset vocabulary

In [62]:
vocabulary = pd.read_csv('vocabulary.tsv',sep="\t")
vocabulary = vocabulary.drop_duplicates(subset=['Word'])
vocabulary = vocabulary.reset_index(drop=True)

In [63]:
vocabulary

,Word,Term_id,Document_List
0,door,term_id_1,['document_1']
1,end,term_id_2,"['document_1', 'document_7']"
2,silent,term_id_3,['document_1']
3,corridor,term_id_4,['document_1']
4,haunt,term_id_5,['document_1']
...,...,...,...
902,protect,term_id_903,['document_19']
903,babi,term_id_904,['document_19']
904,fantin,term_id_905,['document_19']
905,driven,term_id_906,['document_19']


## create the first inverted_list

In [64]:
def make_it_string(word):
    for sym in ["[", "]", "'"]:
        word = word.replace(sym, "")
    word = word.split(", ")
    
    return word

In [65]:
inv_lst = {}
for i, row in vocabulary.iterrows():
    term = vocabulary.at[i, "Term_id"]
    inv_lst[term] = make_it_string(vocabulary.at[i, "Document_List"])

save as csv file

In [66]:
with open('inv_lst.csv', 'w') as f:
    for key in inv_lst.keys():
        f.write("%s:%s\n"%(key,inv_lst[key]))

# Search Engine, 1st version

In [67]:
# open the inv_lst
reader = csv.reader(open('inv_lst.csv', 'r'), delimiter=":")
inv_lst = {}
for row in reader:
    k, v = row
    inv_lst[k] = v

In [68]:
for key in inv_lst:
    inv_lst[key] = make_it_string(inv_lst[key])

In [69]:
# find each term_id
term_id = []
for token in cleanQString:
    term = vocabulary.loc[vocabulary["Word"] == token, "Term_id"].values[0]
    term_id.append(term)

In [70]:
intersection_list = []
for term in term_id:
    if not intersection_list:
        intersection_list = inv_lst[term]
    else:
        intersection_list = set(intersection_list).intersection(set(inv_lst[term]))

In [71]:
new_df = pd.DataFrame(columns=['bookTitle', 'Plot', 'Url'])

for row in intersection_list:
    i = int(row.split("_")[1])
    
    #append row to the dataframe
    new_row = {'bookTitle': df.loc[i, "bookTitle"], 'Plot': df1.loc[i, "Plot"], 'Url': df.loc[i, "Url"]}
    new_df = new_df.append(new_row, ignore_index=True)

In [72]:
new_df

,bookTitle,Plot,Url
0,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,https://www.goodreads.com/book/show/2.Harry_Po...


## RQ[2.2] Conjunctive query & Ranking score

In [73]:
vocabulary

,Word,Term_id,Document_List
0,door,term_id_1,['document_1']
1,end,term_id_2,"['document_1', 'document_7']"
2,silent,term_id_3,['document_1']
3,corridor,term_id_4,['document_1']
4,haunt,term_id_5,['document_1']
...,...,...,...
902,protect,term_id_903,['document_19']
903,babi,term_id_904,['document_19']
904,fantin,term_id_905,['document_19']
905,driven,term_id_906,['document_19']


In [74]:
# open the inv_lst
reader = csv.reader(open('inv_lst.csv', 'r'), delimiter=":")
inv_lst = {}
for row in reader:
    k, v = row
    inv_lst[k] = v

In [75]:
for key in inv_lst:
    inv_lst[key] = make_it_string(inv_lst[key])

In [114]:
import math

inv_lst2 = {}
for i, row in vocabulary.iterrows():
    lst_doc = make_it_string(vocabulary.at[i, 'Document_List'])
    result = []
    for doc in lst_doc:
        number_doc = int(doc.split("_")[1])

        interested_row = df.at[number_doc, "Plot"]
        
        interested_word = vocabulary.at[i, "Word"] #i-th word
        
        tf = interested_row.count(interested_word) / len(interested_row)
        
        idf = math.log(len(df)/len(lst_doc))

        tf_idf = round(tf * idf, 3)
       
        result.append((doc, tf_idf))
        
    inv_lst2[vocabulary.at[i, "Term_id"]] = result

## RQ[2.2.2] SEARCH ENGINE 2

In [79]:
def make_it_string(word):
    for sym in ["[", "]", "'", "(", ")"]:
        word = word.replace(sym, "")
    word = word.split(", ")
    
    return list(word)

In [77]:
query = input("Insert your query string: ")

Insert your query string: door end


In [78]:
cleanQString = cleanQuery(query)
cleanQString

['door', 'end']

In [80]:
# find each term_id
def returnTermId(token):
    return vocabulary.loc[vocabulary["Word"] == token, "Term_id"].values[0]

In [87]:
documents = []
for i, row in df.iterrows():
    print(i)
    tokens = {}
    for token in df.at[i, "Plot"]:
        if token != "nan": #nlp --> wrong the process
            tuple_list_values = inv_lst2[returnTermId(token)]
        
            for x in tuple_list_values:
                if int(x[0].split("_")[1]) == i:
                    tokens[token] = x[1]  
                    break
        
    documents.append(tokens)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [125]:
import math

top_k_documents = [] #similarity
for i, row in df.iterrows():
    card_d_i = 1 / math.sqrt( sum(documents[i-1].values()) )
    # card_q = 1 / math.sqrt(len(cleanQString)) # we suppose this costant to be equal to each document and so we obtain the same order for the cosine similarity
    
    somma = 0
    for token in cleanQString:
        try:
            somma += documents[i-1][token]
        except:
            somma += 0
            
    cosine_similarity = card_d_i * somma
        
    top_k_documents.append([round(cosine_similarity, 2), "document_"+str(i)])

In [126]:
import heapq 
# using heapify to convert list into heap 
heapq.heapify(top_k_documents) 
show_top_k_documents = (heapq.nlargest(20, top_k_documents)) 

In [127]:
new_df = pd.DataFrame(columns=['bookTitle', 'Plot', 'Url', 'Similarity'])

for row in show_top_k_documents:
    i = int(row[1].split("_")[1])
    
    #append row to the dataframe
    new_row = {'bookTitle': df.loc[i, "bookTitle"], 'Plot': df1.loc[i, "Plot"], 'Url': df.loc[i, "Url"], 'Similarity': row[0]}
    new_df = new_df.append(new_row, ignore_index=True)

In [128]:
new_df

,bookTitle,Plot,Url,Similarity
0,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,https://www.goodreads.com/book/show/2.Harry_Po...,0.04
1,The Chronicles of Narnia,"Journeys to the end of the world, fantastic cr...",https://www.goodreads.com/book/show/11127.The_...,0.01
2,The Fault in Our Stars,Despite the tumor-shrinking medical miracle th...,https://www.goodreads.com/book/show/11870085-t...,0.00
3,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,"This four-volume, boxed set contains J.R.R. To...",https://www.goodreads.com/book/show/30.J_R_R_T...,0.00
4,Animal Farm,Librarian's note: There is an Alternate Cover ...,https://www.goodreads.com/book/show/170448.Ani...,0.00
5,The Book Thief,Librarian's note: An alternate cover edition c...,https://www.goodreads.com/book/show/19063.The_...,0.00
6,Twilight,About three things I was absolutely positive.F...,https://www.goodreads.com/book/show/41865.Twil...,0.00
7,Pride and Prejudice,Alternate cover edition of ISBN 9780679783268S...,https://www.goodreads.com/book/show/1885.Pride...,0.00
8,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,https://www.goodreads.com/book/show/2657.To_Ki...,0.00
9,Les Misérables,"Victor Hugo's tale of injustice, heroism and l...",https://www.goodreads.com/book/show/24280.Les_...,0.00


## RQ[3.] Define a new score!

In [14]:
query = input("Insert your query string: ")

Insert your query string: Let's do the thing videogame


In [26]:
cleanQString = cleanQuery(query)
cleanQString

["let'", 'thing', 'videogam']

In [27]:
# open the inv_lst
reader = csv.reader(open('inv_lst.csv', 'r'), delimiter=":")
inv_lst = {}
for row in reader:
    k, v = row
    inv_lst[k] = v

In [28]:
for key in inv_lst:
    inv_lst[key] = make_it_string(inv_lst[key])

In [37]:
# find each term_id
term_id = []
for token in cleanQString:
    term = vocabulary.loc[vocabulary["Word"] == token, "Term_id"].values[0]
    term_id.append(term)

In [38]:
intersection_list = []
for term in term_id:
    if not intersection_list:
        intersection_list = inv_lst[term]
    else:
        intersection_list = set(intersection_list).intersection(set(inv_lst[term]))

In [39]:
new_df = pd.DataFrame(columns=['bookTitle', 'Plot', 'Url'])

for row in intersection_list:
    i = int(row.split("_")[1])
    
    #append row to the dataframe
    new_row = {'bookTitle': df.loc[i, "bookTitle"], 'Plot': df1.loc[i, "Plot"], 'Url': df.loc[i, "Url"]}
    new_df = new_df.append(new_row, ignore_index=True)

In [40]:
new_df

,bookTitle,Plot,Url
